In [15]:
"""
Created on Thu Nov 15-01-2024
Pricing of European Call and Put options using the FFT
@author: Magnus Højmark Kristensen
"""

# import matplotlib.pyplot as plt
# import time
# from numba import njit
# import numpy as np # Has to be 1.20 version as np.long was deprecated
# from numpy import sqrt, exp, pi, cos, sin, log, abs


import numpy as np
from numpy import sqrt, exp, pi, cos, sin, log, abs
from numba import njit
import timeit
#import pandas as pd

## Functions

In [7]:
@njit
def Fourier_Heston_Put(S0, K, T, r, 
                    # Heston Model Paramters
                    kappa,      # Speed of the mean reversion 
                    theta,      # Long term mean
                    rho,        # correlation between 2 random variables
                    zeta,       # Volatility of volatility
                    v0,         # Initial volatility 
                    opt_type,   # Option type
                    N = 2048):

    # Characteristic function of the Heston model
    def heston_char(u, kappa, theta, rho, zeta, v0): 
        t0 = 0.0 ;  q = 0.0
        m = log(S0) + (r - q)*(T-t0)
        D = sqrt((rho*zeta*1j*u - kappa)**2 + zeta**2*(1j*u + u**2))
        C = (kappa - rho*zeta*1j*u - D) / (kappa - rho*zeta*1j*u + D)
        beta = ((kappa - rho*zeta*1j*u - D)*(1-exp(-D*(T-t0)))) / (zeta**2*(1-C*exp(-D*(T-t0))))
        alpha = ((kappa*theta)/(zeta**2))*((kappa - rho*zeta*1j*u - D)*(T-t0) - 2*log((1-C*exp(-D*(T-t0))/(1-C))))
        return exp(1j*u*m + alpha + beta*v0)

    # Parameters for the Function to make sure the approximations are correct.
    c1 = log(S0) + r*T + (1 - exp(-kappa*T))*(theta-v0)/(2*kappa) - (1/2)*theta*T
    c2 = (
        (1/(8*kappa**3)) * (zeta*T*kappa*exp(-kappa*T)*(v0-theta)*(8*kappa*rho-4*zeta)
        + kappa*rho*zeta*(1-exp(-kappa*T))*(16*theta-8*v0)
        + 2*theta*kappa*T*(-4*kappa*rho*zeta + zeta**2 + 4*kappa**2)
        + zeta**2*((theta - 2*v0)*exp(-2*kappa*T) + theta*(6*exp(-kappa*T)-7) + 2*v0)
        + 8*kappa**2*(v0-theta)*(1-exp(-kappa*T)))
    )
    a = c1 - 24*sqrt(abs(c2))
    b = c1 + 24*sqrt(abs(c2))

    h       = lambda n : (n*pi) / (b-a) 
    g       = lambda n : (exp(a) - (K/h(n))*sin(h(n)*(a - log(K))) - K*cos(h(n)*(a - log(K)))) / (1 + h(n)**2)
    g0      = K*(log(K) - a - 1) + exp(a)
    

    F = g0 
    for n in range(1, N+1):
        h_n = h(n)
        F += 2*heston_char(h_n, kappa, theta, rho, zeta, v0) * exp(-1j*a*h_n) * g(n)

    F = exp((-r*T))/(b-a) * np.real(F)
    F = F if opt_type == 'p' else F + S0 - K*exp(-r*T)
    return F if F > 0 else 0

## Parameters

In [16]:
args = {
'S0'      : 100.0 ,
# 'K'       : 100.0 ,
'r'       : 0.05 ,
# 'T'       : 5.0 ,

'kappa' : 1.5768,   # rate of mean reversion of variance process
'theta' : 0.0398,   # long-term mean variance
'zeta' : 0.3,       # volatility of volatility
'rho' : -0.5711,    # correlation between variance and stock process
'v0' : 0.1,         # initial variance

'N' : 96,
'opt_type' : 'p'
}

Ns = [8, 16,32, 64, 256]
df = {'N' : Ns, 'avg_residuals' : [], 'Ex. Time' : []}

Ks = np.linspace(0,300, 50)[1:]
Ts = np.array([1/12, 2/12, 3/12, 4/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
Ts = np.linspace(0,1, 50)
Ts = np.logspace(np.log10(1/365),np.log10(1), 50)
Ts

array([0.00273973, 0.00309029, 0.0034857 , 0.00393172, 0.0044348 ,
       0.00500225, 0.00564231, 0.00636428, 0.00717862, 0.00809715,
       0.00913322, 0.01030186, 0.01162003, 0.01310687, 0.01478396,
       0.01667564, 0.01880937, 0.02121612, 0.02393082, 0.02699288,
       0.03044675, 0.03434256, 0.03873685, 0.04369342, 0.0492842 ,
       0.05559035, 0.0627034 , 0.0707266 , 0.07977641, 0.08998418,
       0.10149809, 0.11448526, 0.12913419, 0.14565753, 0.16429511,
       0.18531745, 0.20902971, 0.23577606, 0.26594474, 0.29997364,
       0.3383567 , 0.38165106, 0.43048514, 0.48556777, 0.54769849,
       0.61777913, 0.69682693, 0.78598927, 0.88656036, 1.        ])

## Results

In [23]:
# Compute computational efficiency using the IPython shell.
%timeit Fourier_Heston_Put(100.0, 100.0, 1.0, 0.05, 1.5768, 0.0398, -0.5711, 0.3, 0.1, 'p', N = 1012)

# Compute prices and save them
# Initialize storage
results = {
        'K'                 : [], 
        'T'                 : [],
        'Price Heston'      : [],
        }
for K in Ks:
    for T in Ts:
        args['K'] = K
        args['T'] = T
        results['K'].append( K )
        results['T'].append( T )
        results['Price Heston'].append( Fourier_Heston_Put(**args) )

701 µs ± 218 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
results



{'K': [6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.122448979591836,
  6.1224